In [1]:
from PIL import Image
from ultralytics import YOLO
from random import shuffle
import random
from glob import glob
import cv2
import PIL
import os

random.seed(1)

In [2]:
from PIL import ImageFont
# use a bitmap font
# use a truetype font


In [58]:
def format_class_names(result):
    class_name_dict = result.names
    for key, value in class_name_dict.items():
        class_name_dict[key] = value.replace('_', ' ')
    return(class_name_dict)


def plot_predictions_on_img(result, predict_img_dir):
    image = cv2.imread(result.path) 
    boxes = result.boxes
    class_name_dict = format_class_names(result)
    for box in boxes:
        class_number = box.cls.cpu().detach().tolist()[0]
        class_name = class_name_dict[class_number] 

        xmin, ymin, xmax, ymax = box.xyxy[0]

        conf = box.conf.cpu().detach().tolist()[0]
        start_point = (int(xmin), int(ymin))
        end_point = (int(xmax), int(ymax))
        
        ## Use simsum.ttc to write Chinese.
        fontpath = "SegUIVar.ttf"     
        font = ImageFont.truetype(fontpath, 32)
        
        img_pil = Image.fromarray(img)
        draw = ImageDraw.Draw(img_pil)
        draw.text((50, 100),  "hihi", font = font, fill = (b, g, r, a))
        img = np.array(img_pil)

        image = cv2.rectangle(image, start_point, end_point, color=(0,0,0), thickness=1)
        image = cv2.putText(image, f"{class_name}: {conf:0.2f}",(int(xmin), int(ymin) - 10),
                            font = font,fontScale = 0.6,color = (255, 255, 255),
                            thickness=2)

    window_name = 'image'
    # Using cv2.imshow() method 
    # Displaying the image 
    #cv2.imshow(window_name, image) 
    cv2.imwrite(os.path.join(predict_img_dir, os.path.basename(result.path)), image)

In [20]:
import numpy as np
from PIL import ImageFont, ImageDraw, Image
import cv2
import time

{0: 'closed roof tank', 1: 'external floating roof tank', 2: 'spherical tank'}

In [25]:
import cv2
import os
from PIL import Image, ImageDraw, ImageFont
import numpy as np

def format_class_names(result):
    class_name_dict = result.names
    for key, value in class_name_dict.items():
        class_name_dict[key] = value.replace('_', ' ')
    return(class_name_dict)

def plot_predictions_on_img(result, predict_img_dir, box_colors):
    image = cv2.imread(result.path)
    boxes = result.boxes
    class_name_dict = format_class_names(result)

    for box in boxes:
        class_number = box.cls.cpu().detach().tolist()[0]
        class_name = class_name_dict[class_number]
        class_color = box_colors[class_number]

        xmin, ymin, xmax, ymax = box.xyxy[0]  
        conf = box.conf.cpu().detach().tolist()[0]

        start_point = (int(xmin), int(ymin)) 
        end_point = (int(xmax), int(ymax))

        # Use simsum.ttc font
        fontpath = "SegUIVar.ttf"     
        font = ImageFont.truetype(fontpath, 12)

        img_pil = Image.fromarray(image)
        draw = ImageDraw.Draw(img_pil)
        draw.text((int(xmin), int(ymin) - 10), f"{class_name}: {conf:0.2f}", font=font)#, fill=(b, g, r, a)) 
        image = np.array(img_pil)

        cv2.rectangle(image, start_point, end_point, color=class_color, thickness=2)

        #cv2.putText(image, ,
        #            (int(xmin), int(ymin) - 10), 
        #            font=font,
        #            fontScale=0.6,
        #            color=(255, 255, 255),
        #            thickness=2)

        # Using cv2.imshow() method 
        # Displaying the image
        #cv2.imshow(window_name, image)

        cv2.imwrite(os.path.join(predict_img_dir, os.path.basename(result.path)), image)

In [4]:
model_path="/work/csr33/object_detection/runs/detect/train_w_tuned_hyperparameters3/weights/best.pt"
# Load a pretrained YOLOv8n model
model = YOLO(model_path)

In [5]:
test_img_dir = "/work/csr33/ast/datasets/yolo/images/train"
test_img_paths = glob(test_img_dir+"/*")

In [6]:
indicies = [i for i in range(len(test_img_paths))]
shuffle(indicies)
images = [test_img_paths[i] for i in indicies]

In [7]:
rand_indices = random.sample(range(len(images)), 10)
random_imgs = [images[i] for i in rand_indices]

In [8]:
results = model(random_imgs, conf=0.5)  # results list


0: 640x640 1 spherical_tank, 1: 640x640 9 closed_roof_tanks, 2: 640x640 4 closed_roof_tanks, 3: 640x640 7 closed_roof_tanks, 4: 640x640 2 closed_roof_tanks, 5: 640x640 4 external_floating_roof_tanks, 6: 640x640 1 closed_roof_tank, 7: 640x640 1 closed_roof_tank, 8: 640x640 17 closed_roof_tanks, 9: 640x640 2 closed_roof_tanks, 258.0ms
Speed: 3.5ms preprocess, 25.8ms inference, 5.8ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
plot_predictions_on_img(result, predict_img_dir)

In [26]:
# Run inference on 'bus.jpg' 
predict_img_dir = "/hpc/home/csr33/prediction_imgs"#"/hpc/group/borsuklab/csr33/object_detection/prediction_imgs"
box_colors = [(255,0,0,0.8), (255,0,168,0.8), (255,0,81,0.8)]

for result in results:
    plot_predictions_on_img(result, predict_img_dir, box_colors)
    #plot_predictions_on_img(result, predict_img_dir)
    #result.show()  # display to screen
    #result.save(filename='result.jpg')  # save to disk

TypeError: list indices must be integers or slices, not float